# Gaussian process classifier for MNIST

This notebook trains and evaluates a random Fourier feature based Gaussian process classifier model (RFFGPC) on the MNIST dataset.
By changing the parameters and repeating the learning and evaluation of this notebook, the following results can be obtained.
Note that the accuracy may vary slightly due to the influence of random numbers.
The results shown in the figure are run on the author's computer with the random number seed fixed as `rfflearn.seed(111)`.

<div align="center">
    <img src="./figures/Inference_time_and_acc_on_MNIST_gpc.svg" width="640" alt="Inference time and acc on MNIST"/>
</div>

In [1]:
import numpy as np
import sklearn.datasets

# Import rfflearn.
import rfflearn.cpu as rfflearn

# If you want to enable GPU, please import rfflearn like the following instead.
#import rfflearn.gpu as rfflearn

## Prepare MNIST dataset

### Load MNIST dataset

Load the MNIST dataset using `sklearn.fetch_openml` and standardize it.
Also the loaded label data is expressed as a string, so it is converted to an integer type.

In [2]:
%%time

# Load MNIST.
Xs, ys = sklearn.datasets.fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False, data_home="./scikit_learn_data")

# Split to training and test data.
Xs_train, Xs_test, ys_train, ys_test = sklearn.model_selection.train_test_split(Xs, ys, test_size=10000, shuffle=False)

# Standardize the data (convert the input data range from [0, 255] to [0, 1]).
Xs_train = Xs_train.astype(np.float64) / 255.0
Xs_test  = Xs_test.astype(np.float64)  / 255.0

# Convert the string label to integer label.
ys_train = ys_train.astype(np.int32)
ys_test  = ys_test.astype(np.int32)

CPU times: total: 5.05 s
Wall time: 18.4 s


### Data dimension reduction

Reduce data dimension using PCA.
This step is not necessary, but contribute to the test accuracy.

In [3]:
%%time

dim_pca = 128

# Create matrix for principal component analysis.
_, V = np.linalg.eig(Xs_train.T @ Xs_train)
T = np.real(V[:, :dim_pca])

CPU times: total: 2.53 s
Wall time: 603 ms


## Train and evaluate a GPC model

### Instanciate GPC model

Instanciate one of the following GPC model:

* `RFFGPC`: Gaussian process classifier with random Fourier features.
* `ORFGPC`: Similar to `RFFGPC`, but orthogonal random features are used.
* `QRFGPC`: Similar to `RFFGPC`, but quasi-random numbers are used.

In [4]:
# Gaussian process classifier with random Fourier features.
gpc = rfflearn.RFFGPC(dim_kernel=10000, std_kernel=0.10, std_error=0.1)

# Gaussian process classifier with orthogonal random features.
# gpc = rfflearn.ORFGPC(dim_kernel=10000, std_kernel=0.10, std_error=0.1)

# Gaussian process classifier with quasi-random Fourier features.
# gpc = rfflearn.QRFGPC(dim_kernel=10000, std_kernel=0.10, std_error=0.1)

### Train the model

Train the Gaussian process model.

In [5]:
%%time

rfflearn.seed(111)

# Train GPC.
gpc.fit(Xs_train @ T, ys_train)

CPU times: total: 9min 3s
Wall time: 1min 30s


### Evaluate on the test data

In [6]:
%%time

# Calculate score for test data.
score = 100 * gpc.score(Xs_test @ T, ys_test)
print(f"Score = {score:.2f} [%]")

Score = 98.12 [%]
CPU times: total: 5.14 s
Wall time: 3.11 s
